In [ ]:

import numpy as np

import torch
import pytorch3d
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes #,knn_points, estimate_pointcloud_normals, knn_gather

import trimesh

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')


In [ ]:
from ops.utils import *

In [ ]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

For Mesh

In [ ]:
from ops.mesh_geometry import *

In [ ]:
trg_mesh = load_objs_as_meshes(["kitty.obj"], device=device)

In [ ]:
points_np = trg_mesh.verts_packed().cpu().numpy()
faces_np = trg_mesh.faces_packed().cpu().numpy()
mesh_np = trimesh.Trimesh(vertices=points_np, faces=faces_np)

In [ ]:
curvature, euler = gaussian_curvature(trg_mesh, return_topology=True)

print('Euler:', euler)



In [ ]:
dual_area = dual_area_vertex(trg_mesh)
curvature_vertex = curvature.view(1,-1)*dual_area.view(1,-1)
# discrete Gauss-Bonnet theorem
print("Gauss-Bonnet theorem: integral of gaussian_curvature - 2*pi*X = ",(curvature_vertex).sum().cpu().numpy() - 2*np.pi*mesh_np.euler_number)

In [ ]:
pl = pv.Plotter(notebook=True)

rescaled_curv = torch.tanh(curvature_vertex*100)
pl.add_mesh(mesh_np, scalars=rescaled_curv.cpu().numpy(), show_edges=True, edge_color='white', cmap='viridis', clim=[-1,1], line_width=0.001)

pl.show()

For PCL

In [ ]:
from ops.pcl_geometry import *

In [ ]:
device = torch.device("cuda:0")


trg_mesh = load_objs_as_meshes(["kitty.obj"], device=device)

# trg_mesh = ico_sphere(5, device)

trg_mesh = normalize_mesh(trg_mesh)

In [ ]:
num_samples = 100000
pointscloud, normals_gt = sample_points_from_meshes(trg_mesh, num_samples, return_normals=True)

In [ ]:
pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 10)

gaussian_curvature = pcl_with_frames.gaussian_curvature().view(-1)

# trace of the Weingarten map
mean_curvature = pcl_with_frames.weingarten_fields().diagonal(offset=0, dim1=-1, dim2=-2).mean(-1).view(-1)

area = pcl_with_frames.local_voronoi_area().view(-1)



In [ ]:
for i in range(3,9):
    pcl_with_frames = Differentiable_Global_Geometry_PointCloud(pointscloud, k = 10*i)
    print('Estimated Genus',  1-torch.round(pcl_with_frames.differentiable_euler_number()).item()/2)

    print('Estimated euler',  pcl_with_frames.differentiable_euler_number())


In [ ]:
pl = pv.Plotter(notebook=True)
rescaled_curv = torch.tanh(gaussian_curvature*area*200)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[0], scalars=rescaled_curv.detach().cpu().numpy(), point_size=5,cmap="viridis")

pl.show()

In [ ]:
pl = pv.Plotter(notebook=True)
rescaled_curv = torch.tanh(mean_curvature*area*2000)
pl.add_points(pcl_with_frames.pointscloud.detach().cpu().numpy()[0], scalars=rescaled_curv.detach().cpu().numpy(), point_size=5,cmap="viridis")
pl.show()

In [ ]:
pl